# Práctica 6

Ahora es tu turno de trabajar con embeddings en distintos contextos.

## Ejercicio obligatorio (5 puntos)

El ejercicio que tenéis que realizar obligatoriamente consiste en elegir uno de los datasets del [UCI Machine learning repository](https://archive.ics.uci.edu/ml/datasets.php) y construir un modelo siguiendo las instrucciones proporcionadas en el notebook de instrucciones. La única restricción es que el dataset debe contener tanto datos numéricos como categóricos.

## Ejercicios opcionales
- Recupera lo que viste en la asignatura de aprendizaje automático II para construir modelos usando técnicas clásicas y compara el modelo que has creado en el ejercicio obligatorio con al menos los siguientes modelos KNN, SVM, Decision Tree y Random Forest. (2 puntos)
- En el notebook practica6Opcional.ipynb se incluye un notebook con varios ejercicios para trabajar con word embeddings. (3 puntos)


In [4]:
!pip install fastai -Uq

     |████████████████████████████████| 197 kB 4.4 MB/s 
     |████████████████████████████████| 60 kB 5.7 MB/s 


In [1]:
from fastai.tabular.all import *
import pandas as pd
import random

In [2]:
from sklearn.model_selection import train_test_split

# Intento con Post-Operative

In [12]:
columns = ['L-CORE','L-SURF','L-02','L-BP','SURF-STBL','CORE-STBL','BP-STBL','COMFORT','decision ADM-DECS']

In [13]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/postoperative-patient-data/post-operative.data -O postOperative.csv

--2022-05-15 18:50:45--  https://archive.ics.uci.edu/ml/machine-learning-databases/postoperative-patient-data/post-operative.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4379 (4.3K) [application/x-httpd-php]
Saving to: ‘postOperative.csv’

postOperative.csv   100%[===================>]   4.28K  --.-KB/s    in 0s      

2022-05-15 18:50:46 (139 MB/s) - ‘postOperative.csv’ saved [4379/4379]



In [26]:
df = pd.read_csv('postOperative.csv',header=None,names=columns)
df = df.where(df != '?', np.NaN)
df = df.dropna() 
df = df.astype({"COMFORT": float})

df_train, df_test = train_test_split(df, test_size=0.2)
dep_var = 'decision ADM-DECS'
procs = [Categorify, FillMissing,Normalize]
cont,cat = cont_cat_split(df_train, 1, dep_var=dep_var)

In [27]:
cont

['COMFORT']

In [28]:
cat

['L-CORE', 'L-SURF', 'L-02', 'L-BP', 'SURF-STBL', 'CORE-STBL', 'BP-STBL']

In [29]:
dlsTrain = TabularDataLoaders.from_df(df_train,'.',procs=procs,
                                cat_names=cat,cont_names=cont,y_names='decision ADM-DECS',
                                valid_idx=random.sample(range(0,len(df_train)),int(len(df_train)*0.2)),bs=64)

In [30]:
learn = tabular_learner(dlsTrain, layers=[500,250],metrics=[accuracy])

In [31]:
learn.model

TabularModel(
  (embeds): ModuleList(
    (0): Embedding(4, 3)
    (1): Embedding(4, 3)
    (2): Embedding(3, 3)
    (3): Embedding(4, 3)
    (4): Embedding(3, 3)
    (5): Embedding(4, 3)
    (6): Embedding(4, 3)
  )
  (emb_drop): Dropout(p=0.0, inplace=False)
  (bn_cont): BatchNorm1d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layers): Sequential(
    (0): LinBnDrop(
      (0): Linear(in_features=22, out_features=500, bias=False)
      (1): ReLU(inplace=True)
      (2): BatchNorm1d(500, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): LinBnDrop(
      (0): Linear(in_features=500, out_features=250, bias=False)
      (1): ReLU(inplace=True)
      (2): BatchNorm1d(250, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (2): LinBnDrop(
      (0): Linear(in_features=250, out_features=3, bias=True)
    )
  )
)

In [32]:
learn.lr_find()

ZeroDivisionError: ignored

In [20]:

learn = tabular_learner(dlsTrain, layers=[500,250],metrics=[accuracy])
learn.fit(10, 1e-3)
dfAll = pd.concat([df_train,df_test])
dlsTest = TabularDataLoaders.from_df(dfAll,'.',procs=procs,
                                cat_names=cat,cont_names=cont,y_names='decision ADM-DECS',
                                valid_idx=list(range(len(df_train)+1,len(dfAll))),bs=64)
learn.dls=dlsTest
learn.validate()

epoch,train_loss,valid_loss,accuracy,time
0,nan,1.372723,0.692308,00:00
1,nan,1.372723,0.692308,00:00
2,nan,1.372723,0.692308,00:00
3,nan,1.372723,0.692308,00:00
4,nan,1.372723,0.692308,00:00
5,nan,1.372723,0.692308,00:00
6,nan,1.372723,0.692308,00:00
7,nan,1.372723,0.692308,00:00
8,nan,1.372723,0.692308,00:00
9,nan,1.372723,0.692308,00:00


/usr/local/lib/python3.7/dist-packages/fastprogress/fastprogress.py:74: UserWarning: Your generator is empty.
  warn("Your generator is empty.")


(#2) [1.3768978118896484,0.0]

# Intento con Abalone

In [3]:
columns = ['Sex','Length','Diameter','Height','Whole weight','Shucked weight','Viscera weight','Shell weight','Rings']

In [4]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/abalone/abalone.data -O data.csv

--2022-05-15 18:45:48--  https://archive.ics.uci.edu/ml/machine-learning-databases/abalone/abalone.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 191873 (187K) [application/x-httpd-php]
Saving to: ‘data.csv’

data.csv            100%[===================>] 187.38K   475KB/s    in 0.4s    

2022-05-15 18:45:50 (475 KB/s) - ‘data.csv’ saved [191873/191873]



In [5]:
df = pd.read_csv('data.csv',header=None,names=columns)

In [79]:
df.head(5)

,Sex,Length,Diameter,Height,Whole weight,Shucked weight,Viscera weight,Shell weight,Rings
0,M,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15
1,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7
2,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9
3,M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10
4,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7


In [81]:
df["Rings"].value_counts()

9     689
10    634
8     568
11    487
7     391
12    267
6     259
13    203
14    126
5     115
15    103
16     67
17     58
4      57
18     42
19     32
20     26
3      15
21     14
23      9
22      6
27      2
24      2
1       1
26      1
29      1
2       1
25      1
Name: Rings, dtype: int64

In [82]:
def myfunc(x):
  if x >= 0 and x < 5:
    return("[1-4]")
  if x >=5  and x < 10:
    return("[5-9]")
  if x >=10 and x < 15:
    return("[10-14]")
  if x >=15 and x < 20:
    return("[15-19]")
  if x >=20 and x < 25:
    return("[20-24]")
  if x >=25 and x < 30:
    return("[25-29]")

In [83]:
df["Rings"] = df["Rings"].apply(myfunc)

In [84]:
df.head(5)

,Sex,Length,Diameter,Height,Whole weight,Shucked weight,Viscera weight,Shell weight,Rings
0,M,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,[15-19]
1,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,[5-9]
2,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,[5-9]
3,M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,[10-14]
4,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,[5-9]


In [6]:
df_train, df_test = train_test_split(df, test_size=0.2)

In [86]:
df_train.head(5)

,Sex,Length,Diameter,Height,Whole weight,Shucked weight,Viscera weight,Shell weight,Rings
2460,M,0.445,0.350,0.140,0.5905,0.2025,0.1580,0.1900,[10-14]
3861,F,0.630,0.475,0.175,1.4230,0.4155,0.3385,0.4900,[10-14]
2647,I,0.500,0.390,0.125,0.5830,0.2940,0.1320,0.1605,[5-9]
1666,F,0.605,0.470,0.155,0.9740,0.3930,0.2240,0.3345,[5-9]
2717,I,0.345,0.255,0.095,0.1830,0.0750,0.0385,0.0600,[5-9]


In [7]:
dep_var = 'Rings'

In [8]:
procs = [Categorify, FillMissing,Normalize]

In [9]:
cont,cat = cont_cat_split(df_train, 1, dep_var=dep_var)

In [10]:
cat

['Sex']

In [11]:
cont

['Length',
 'Diameter',
 'Height',
 'Whole weight',
 'Shucked weight',
 'Viscera weight',
 'Shell weight']

In [92]:
dlsTrain = TabularDataLoaders.from_df(df_train,'.',procs=procs,
                                cat_names=cat,cont_names=cont,y_names='Rings',
                                valid_idx=random.sample(range(0,len(df_train)),int(len(df_train)*0.2)),bs=64)

In [93]:
learn = tabular_learner(dlsTrain, layers=[500,250],metrics=[])

In [94]:
learn.model

TabularModel(
  (embeds): ModuleList(
    (0): Embedding(4, 3)
  )
  (emb_drop): Dropout(p=0.0, inplace=False)
  (bn_cont): BatchNorm1d(7, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layers): Sequential(
    (0): LinBnDrop(
      (0): Linear(in_features=10, out_features=500, bias=False)
      (1): ReLU(inplace=True)
      (2): BatchNorm1d(500, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): LinBnDrop(
      (0): Linear(in_features=500, out_features=250, bias=False)
      (1): ReLU(inplace=True)
      (2): BatchNorm1d(250, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (2): LinBnDrop(
      (0): Linear(in_features=250, out_features=6, bias=True)
    )
  )
)

In [95]:
learn.lr_find()

KeyboardInterrupt: ignored

In [ ]:
learn.fit(10, 1e-3)

In [ ]:
dfAll = pd.concat([df_train,df_test])
dlsTest = TabularDataLoaders.from_df(dfAll,'.',procs=procs,
                                cat_names=cat,cont_names=cont,y_names='Rings',
                                valid_idx=list(range(len(df_train)+1,len(dfAll))),bs=64)

In [ ]:
learn.dls=dlsTest

In [ ]:
learn.validate()

In [ ]:
learn.predict(df_test.iloc[0][:-1])

Al finalizar, recuerda guardar los cambios en GitHub utilizando la opción Archivo -> Guardar una copia en GitHub.
